<a href="https://colab.research.google.com/github/LinaMariaCastro/curso-ia-para-economia/blob/main/clases/4_Aprendizaje_no_supervisado/2_Taller_Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Inteligencia Artificial con Aplicaciones en Economía I**

- 👩‍🏫 **Profesora:** [Lina María Castro](https://www.linkedin.com/in/lina-maria-castro)  
- 📧 **Email:** [lmcastroco@gmail.com](mailto:lmcastroco@gmail.com)  
- 🎓 **Universidad:** Universidad Externado de Colombia - Facultad de Economía

# **Taller: Análisis de Patrones de Consumo Internacional con Apriori**

**IMPORTANTE**: Guarda una copia de este notebook en tu Google Drive o computador.

**Taller en parejas**

**Nombres estudiantes:**

-
-

**Forma de entrega**

Jupyter Notebook publicado en su cuenta de Github con el nombre “Taller_Apriori_apellidos_estudiantes.ipynb”.

**Plazo de entrega**

Hoy, 16 de octubre, máximo a las 11:59 p.m., debes enviar link del notebook al correo lina.castro6@uexternado.edu.co, de lo contrario, no será tenido en cuenta.

**Instrucciones Generales**

Completa el código en las celdas marcadas con `### TU CÓDIGO AQUÍ ###`. Puedes añadir más celdas si lo requieres.

**Caso de Estudio: Consultoría para el PNUD**

**Contexto:** Una firma multinacional de e-commerce, "Global Retail Inc.", te ha contratado como consultor de datos. La empresa opera en múltiples países y ha notado que sus ventas y la efectividad de sus campañas de marketing varían significativamente entre regiones. Su hipótesis es que los patrones de compra y las asociaciones de productos son diferentes en cada mercado.

**Tu Misión:** Analizar el historial de transacciones de la empresa para descubrir y comparar las reglas de asociación de productos para dos de sus mercados más grandes: el Reino Unido y Alemania. Tu objetivo final es entregar recomendaciones de negocio accionables (ej. estrategias de cross-selling, promociones personalizadas) basadas en los patrones de consumo que descubras en cada país.

**Dataset:** Online Retail Data Set del repositorio de UCI. Encuentra mayor información en: https://archive.ics.uci.edu/dataset/352/online+retail

## Ejercicio 1: Configuración Inicial, Carga y Exploración de Datos

1.1 Importa las librerías necesarias

In [ ]:
### TU CÓDIGO AQUÍ ###

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Configuraciones de visualización
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.2f}'.format

1.2 Carga el dataset "retail_top30_products.csv" que se encuentra en el repositorio del curso, carpeta "datasets".

In [ ]:
### TU CÓDIGO AQUÍ ###


In [ ]:
print("Dimensiones del DataFrame:")
print(df.shape)

In [ ]:
print("\nInformación general del DataFrame:")
df.info()

1.3 Revisa si hay valores nulos en alguna columna y cuántos son

In [ ]:
### TU CÓDIGO AQUÍ ###


1.4 Genera las estadísticas descriptivas de las variables numéricas

In [ ]:
### TU CÓDIGO AQUÍ ###


1.5 Observando las salidas del ejercicio anterior, ¿qué problemas potenciales identificas en las columnas Description, CustomerID y Quantity? ¿Ves algo inusual en los precios (UnitPrice)?

## Ejercicio 2: Limpieza y Preprocesamiento de Datos

Los datos del mundo real rara vez son perfectos. Antes de cualquier análisis, debemos "sanear" nuestro dataset. Completa el código en cada paso según las instrucciones.

2.1 **Manejo de Valores Nulos**: Las transacciones sin un CustomerID no son útiles para nosotros, ya que no podemos agrupar las compras de un cliente específico.

In [ ]:
# TAREA: Elimina todas las filas donde 'CustomerID' es nulo.
### TU CÓDIGO AQUÍ ###


In [ ]:
# El tipo de dato de CustomerID debe ser entero
### TU CÓDIGO AQUÍ ###


2.2 **Limpieza de Descripciones de Productos** Las descripciones pueden tener espacios en blanco al inicio o al final que podrían hacer que un mismo producto se cuente como dos diferentes.

In [ ]:
# TAREA: Limpia la columna 'Description' eliminando espacios extra al inicio y al final.
### TU CÓDIGO AQUÍ ###


2.3 **Filtrado de Transacciones Anómalas**: Las facturas (InvoiceNo) que empiezan con 'C' indican una cancelación. Estas no son compras reales y deben ser eliminadas. Del mismo modo, las cantidades (Quantity) negativas representan devoluciones.

In [ ]:
# TAREA: Elimina las filas que correspondan a cancelaciones.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Elimina las filas con cantidades negativas.
### TU CÓDIGO AQUÍ ###


In [ ]:
# NO MODIFICAR ESTA CELDA
assert df_limpio['Quantity'].min() > 0, "Error: Todavía hay cantidades negativas."
assert df_limpio['CustomerID'].isnull().sum() == 0, "Error: Aún hay valores nulos en CustomerID."
assert df_limpio[df_limpio['InvoiceNo'].str.contains('C')].empty, "Error: Todavía hay facturas de cancelación."
print("¡La limpieza básica ha sido exitosa!")

## Ejercicio 3: Análisis Comparativo por País

Ahora que los datos están limpios, vamos a segmentarlos y a aplicar el algoritmo Apriori para encontrar los patrones de compra en el Reino Unido y Alemania.

**Preparación de la Cesta de Mercado (Función)**

La siguiente función toma un dataframe, lo agrupa por factura y descripción, y lo transforma en el formato de matriz binaria que necesita el algoritmo Apriori. Estudia esta función, no necesitas modificarla.

In [ ]:
# NO MODIFICAR ESTA CELDA
def preparar_cesta(dataframe, pais):
    """Filtra por país y prepara la matriz de transacciones."""

    # Filtrar por el país de interés
    df_pais = dataframe[dataframe['Country'] == pais]

    # Crear la cesta: agrupar productos por factura
    cesta = (df_pais.groupby(['InvoiceNo', 'Description'])['Quantity']
             .sum().unstack().reset_index().fillna(0)
             .set_index('InvoiceNo'))

    # Convertir todas las cantidades positivas a 1 y todo lo demás a 0
    def encode_units(x):
        if x <= 0:
            return 0
        if x >= 1:
            return 1

    cesta_encoded = cesta.applymap(encode_units)
    return cesta_encoded

3.1 Análisis para el Reino Unido (United Kingdom)

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones del Reino Unido.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Aplica el algoritmo apriori para encontrar itemsets con un soporte mínimo de 2%.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Genera las reglas de asociación. Queremos reglas con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###


In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###


3.3 Observa las 3 reglas con el Lift más alto para el Reino Unido (1, 3 y 5). **Interprétalas:** ¿Qué te dicen estas asociaciones? ¿Qué tipo de productos son?

3.4 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

3.5 **Recomendación de Negocio:** Basado en estas reglas, ¿qué promoción o estrategia de venta específica podrías sugerir para el mercado del Reino Unido?

3.6 Análisis para Alemania (Germany)

In [ ]:
# TAREA: Usa la función preparar_cesta para obtener la matriz de transacciones de Alemania.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Aplica el algoritmo apriori con un soporte mínimo del 2%.
### TU CÓDIGO AQUÍ ###


In [ ]:
# TAREA: Genera las reglas de asociación con un Lift mayor a 3.
### TU CÓDIGO AQUÍ ###


In [ ]:
# Ordena las reglas por Lift y Confianza de mayor a menor
### TU CÓDIGO AQUÍ ###


3.7 Observa las 3 reglas con el Lift más alto para Alemania (1, 3 y 5). **Interprétalas:** ¿Qué patrones de consumo específicos del mercado alemán revelan estas reglas? ¿Son diferentes a los del Reino Unido?



3.8 Interpreta el Soporte para el antecedente y el consecuente, la Confianza y el Lift

3.9 **Recomendación de Negocio:** ¿Qué campaña de marketing (diferente a la del Reino Unido) podrías diseñar para los clientes alemanes?